In [1]:
import pandas as pd
import os

In [2]:
DATASET = os.path.join(
    "..","data"
)
CHENNAI_DATASET = os.path.join(DATASET,"Chennai_data")

RESULTS = os.path.join(
    "..","results","relative_abundance",
)
os.makedirs(RESULTS, exist_ok = True)

TOP = 4

In [3]:
## Load the Metadata and add the selection site 
metadata_df = pd.read_csv(os.path.join(CHENNAI_DATASET, 'metadata_processed.csv'))
metadata_df.set_index('Samples',inplace = True)
metadata_df.head()

,DNA/RNA,sample type(N/P/E),location name,sublocation,select object/sampling place,select surface material,select ground level,Traffic at the sampling location,temperature,date and time,Sampling Date,Month,Year,is_kiosk,is_Rod
Samples,,,,,,,,,,,,,,,
368257968,DNA,E,Washermenpet,Metro,Bannister,Metal,NaN,NaN,NaN,12:37,8/12/2021,Dec,2021,Other,Other
368273379,DNA,E,Washermenpet,Train,Rod,Metal,NaN,NaN,NaN,12:44,8/12/2021,Dec,2021,Other,Rod
368281532,DNA,E,Central,Metro,Ticket Counter,Ceramic,NaN,NaN,NaN,12:57,8/12/2021,Dec,2021,Other,Other
368281511,DNA,E,Central,Metro,Kiosk,Glass,NaN,NaN,NaN,13:00,8/12/2021,Dec,2021,Kiosk,Other
368281318,DNA,E,Central,Metro,Ticket Counter,Ceramic,NaN,NaN,NaN,13:17,8/12/2021,Dec,2021,Other,Other


In [4]:

def max_col(row):
    return row.sort_values(ascending = False).sum()

"""
find_top(df)

FInd the top n columns and sum rest into others column : Note you have to et the row ID as index and cannot have any 
metadata
"""
def find_top(df, top):
    top_taxa = set()
    for i, row in df.iterrows():
        top_taxa = top_taxa.union(set(row.sort_values(ascending = False).index[:top].values.tolist()))
    return df[list(top_taxa)]


In [7]:
df.head()

,Actinobacteria,Gammaproteobacteria,Malasseziomycetes,Alphaproteobacteria,Bacilli,Others,Betaproteobacteria,Flavobacteriia
Samples,,,,,,,,
368257238,0.449458,0.190281,0.123508,0.080857,0.068885,0.049255,0.023552,0.014204
368257968,0.557205,0.173626,0.019108,0.104927,0.061310,0.040628,0.033007,0.010188
368258054,0.389156,0.166062,0.003267,0.214565,0.036121,0.084555,0.087045,0.019228
368259600,0.402575,0.332831,0.017402,0.112028,0.047170,0.048121,0.031861,0.008011
368259989,0.498228,0.183920,0.013541,0.137151,0.051130,0.053799,0.034523,0.027709


In [8]:
taxa_list = ['class']
for Taxa in taxa_list:
    df = pd.read_csv(os.path.join(CHENNAI_DATASET, f'bracken_{Taxa}_non_human_processed.csv'))
    df.set_index('Samples',inplace = True)
    df = find_top(df, TOP)
    df['Others'] = 1 - df.sum(axis = 1)
    ## get the order of column names
    order = df.iloc[0].sort_values(ascending = False).index
    df = df[order]
    df = pd.merge(df, metadata_df[['select object/sampling place']], left_index=True, right_index=True)
    df.to_csv(os.path.join(RESULTS,f'{Taxa}_with_metadata.csv'))
#df.head()